In [1]:
import jax

from quantum_transformers.datasets import get_imdb_dataloaders
from quantum_transformers.training import train_and_evaluate
from quantum_transformers.transformers import Transformer

data_dir = '/global/cfs/cdirs/m4392/salcc/data'

2023-08-22 00:08:06.583358: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-08-22 00:08:06.583384: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-08-22 00:08:06.583401: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-08-22 00:08:08.172459: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.


Please first ``pip install -U cirq`` to enable related functionality in translation module


In [2]:
for d in jax.devices():
    print(d, d.device_kind)

gpu:0 NVIDIA A100-SXM4-80GB


In [3]:
(imdb_train_dataloader, imdb_valid_dataloader), vocab, tokenizer = get_imdb_dataloaders(batch_size=32, data_dir=data_dir, max_vocab_size=8192, max_seq_len=128)
print(f"Vocabulary size: {len(vocab)}")
first_batch = next(iter(imdb_train_dataloader))
print(first_batch[0][0])
print(' '.join(map(bytes.decode, tokenizer.detokenize(first_batch[0])[0].numpy().tolist())))

Vocabulary size: 8175
[ 103   99   42  170   43  109   54 1453 1782 5719  106   42   54 2817
 6631 1320   17   95 1775  797 1920  102   95  109  138  322  106  168
  368  149   96  966  106  196  175  201   17  104  447 3240  196    5
  797  333    5  104  124  302  119  592   98 1167   17   31  100   18
   33   31  100   18   33   95  293  104   95  112   99  115  267   42
   54 2817 6631 1320   99   95  324   97  101   10   60  854   17   95
  112   99  115  266   17   95  112   99  115 1507   17   95  112  252
   10   61  119  334 7399   98  231  135   95 1775 1140   17  101   10
   60  115  151  706   60 6449 4300   17  234  131   10   60   60 3674
 2676  493]
this is a bad b movie masquerading as a mockumentary . the porn documentary filmmaker in the movie has almost as much screen time and dialog as any other character . that completely destroyed any " documentary feel " that they may have wanted to create . < br / > < br / > the fact that the film is not actually a mockumentary 

In [4]:
model = Transformer(num_tokens=len(vocab), max_seq_len=128, num_classes=2, hidden_size=64, num_heads=2, num_transformer_blocks=4, mlp_hidden_size=32)
train_and_evaluate(model, imdb_train_dataloader, imdb_valid_dataloader, num_classes=2, learning_rate=0.0003, num_epochs=30)

Epoch  30/30: 100%|██████████| 781/781 [00:08<00:00, 91.07batch/s, Loss = 1.5709, AUC = 84.26%]                                           

TOTAL TIME = 333.14s
BEST AUC = 88.86% AT EPOCH 2


In [5]:
model = Transformer(num_tokens=len(vocab), max_seq_len=128, num_classes=2, hidden_size=6, num_heads=2, num_transformer_blocks=4, mlp_hidden_size=3)
train_and_evaluate(model, imdb_train_dataloader, imdb_valid_dataloader, num_classes=2, learning_rate=0.0003, num_epochs=30)

Epoch  30/30: 100%|██████████| 781/781 [00:07<00:00, 100.04batch/s, Loss = 0.7486, AUC = 80.19%]                                          

TOTAL TIME = 321.63s
BEST AUC = 80.63% AT EPOCH 22
